## TF Tutorial

### 基础模型

以后需要用到的一些基础函数，其实跟之前的案例很像，大家在仔细看看。里面加入了tensorflow的一些函数。

In [4]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)

In [3]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)

    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.

    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """

    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)

    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def predict(X, parameters):

    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])

    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}

    x = tf.placeholder("float", [12288, 1])

    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)

    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})

    return prediction

### 一个简单的例子

#### tf 程序运行的步骤

这里我来用tf计算一个损失函数：
$loss = L(\hat y,y)=(\hat y^{(i)}-y^{(i)})^2$

运行一个tf程序的基本步骤：
- 创建张量Tensor（变量）
- 创建节点操作，用来运算张量
- 初始化张量
上面的步骤，是用来创建一个计算图
- 创建Session(会话)，用来运算op
- 运行Session,计算op
上面的步骤，是用来运行这张计算图

具体的coding如下：


In [5]:
# 定义两个常量constant
y_hat = tf.constant(36,name='y_hat')
y = tf.constant(39,name="y")

# 定义一个变量，用来接收计算的值
loss = tf.Variable((y-y_hat)**2,name='loss')

# 用来初始化定义所有变量
init = tf.global_variables_initializer()

# 创建一个Session,用来运行程序
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(loss))

9


#### 理解计算图与Session(会话)

首先看如下例子：
下面的例子中可以看出，c的结果并没有像我们想象的那样计算正确的结果，而是显示关于c的基本信息:
- Mul:表示计算的节点
- shape:表示tensor的size
- dtype:表示tensor的类型

这里仅仅是构建了一张计算图，并没有运行这张计算图
只要通过Session构建会话，run()才能将计算开始运算起来。
再次总结以下：
- 初始化参数变量
- 创建Session
- 使用Session的run函数，完成相关的op操作

In [7]:
# 理解计算图
a = tf.constant(2)
b = tf.constant(10)
print(a)
c = tf.multiply(a,b)
print(c)

Tensor("Const_2:0", shape=(), dtype=int32)
Tensor("Mul_1:0", shape=(), dtype=int32)


In [8]:
# 运行计算图，Sesssion
with tf.Session() as sess:
    print(sess.run(c))


20


#### placeholder占位符的概念

通过占位符，我们可以将数据在使用的时候再指定。
通过使用“feed 字典”来指定，具体函数：feed_dict
一个简单的例子coding如下：
需要说明的如下：

- 定义一个占位符，里面并没有指定任何的数据
- 这个占位符就是告诉我们，我们先留好一个位置，具体谁来坐，等到开会的时候再来定。
- 在需要这个数据的时候，再使用使用feed_dict，组成的字典形式，来告诉tf来使用

In [11]:
# 定义一个占位符，里面并没有指定任何的数据
x = tf.placeholder(tf.int64,name="x")
print(x)
with tf.Session() as sess:
    # 在使用的时候指定对应的数据
    # 使用feed_dict，组成的字典形式
    print(sess.run(2*x,feed_dict={x:3}))

Tensor("x_1:0", dtype=int64)
6


### 线性函数的表示

下面我们正式来开始利用tf来对我们的基础函数的编程，首先是实现线性函数，也就是利用权重和偏置值计算Z.

$
Y = WX + b
$

我们需要用到的tf函数如下：
- tf.matmul() :计算矩阵的乘法
- tf.add() :计算加法
- np.random.randn():初始化一个随机的值

具体的coding如下：





In [13]:
def linear_function():
    
    np.random.seed(1)
    
    # 创建tensor和计算图
    X = tf.constant(np.random.randn(3,1),name="X")
    W = tf.constant(np.random.randn(4,3),name="W")
    b = tf.constant(np.random.randn(4,1),name="b")
    Y = tf.add(tf.matmul(W,X),b)
    
    # 创建会话并运行计算图
    with tf.Session() as sess:
        result =  sess.run(Y)
        
    return result
    
    

In [14]:
print( "result = " + str(linear_function()))

result = [[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]]


In [ ]:
# 个人觉得
def linear_function():
    
    X = tf.placeholder(tf.float32,name="X")
    
    W = tf.Variable(tf.float32,name="W")
    b = tf.Variable(tf.float32,name="b")
    
    Y = tf.add(tf.matmul(W,X),b)
    
     
    return Y

### 计算sigmoid激活函数

这里我们将直接使用tf自带的sigmoid函数，tf.sigmoid(),需要定义的函数参数使用x,使用placeholder来接收传进来的参数。具体的步骤如下：

- 定义tensor: placeholder
- 使用tf.sigmoid(),构建op
- 运行session:记得将对应的数据喂入对应的placeholder中



In [15]:
def sigmoid(z):
    
    x = tf.placeholder(tf.float32,name="x")
    
    sigmoid = tf.sigmoid(x)
    
    with tf.Session() as sess:
        result = sess.run(sigmoid,feed_dict={x:z})
        
    return result

In [18]:
print("simoid(1):" + str(sigmoid(0)))


simoid(1):0.5


### 计算损失函数

对于二分类的损失函数，我们可以定义如下：
![](imgs/2.jpg)

这里我们可以使用tf自带的函数来实现：
`tf.nn.sigmoid_cross_entropy_with_logists(logits = ..., labels = ...)`

其实该公式传入的参数只需要传入`z`它会自动计算`a`,实现的具体公式如下：
![](imgs/3.jpg)

具体coding如下：

In [20]:
def cost(logits,labels):
    
    z = tf.placeholder(tf.float32,name="z")
    y = tf.placeholder(tf.float32,name="y")
    
    cost = tf.nn.sigmoid_cross_entropy_with_logits(
        logits=z,
        labels=y
    )
    
    with tf.Session() as sess:
        cost = sess.run(cost,feed_dict={z:logits,y:labels})
    
    return cost

In [21]:
logits = sigmoid(np.array([0.2,0.4,0.7,0.9]))
cost = cost(logits, np.array([0,0,1,1]))
print ("cost = " + str(cost))

cost = [1.0053872  1.0366408  0.41385433 0.39956617]


### One Hot 编码

就是将具体的数字转换成0，1编码，其过程如下:
![](imgs/4.jpg)

通过tf可以很方面调用其API来实现：
`tf.one_hot(labels,depth,axis)`
下面实例如下：



In [2]:
def one_hot_matrix(labels,C):
    
    C = tf.constant(value=C,name="C")
    
    one_hot_matrix = tf.one_hot(labels,C,axis=0)
    
    with tf.Session() as sess:
        one_hot = sess.run(one_hot_matrix)
        
    return one_hot

In [5]:
labels = np.array([1,2,3,0,2,1])
one_hot = one_hot_matrix(labels, C = 4)
print ("one_hot = " + str(one_hot))

one_hot = [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]]


### 初始化为0或者1

可以直接使用
`tf.ones` 或者 `tf.zeros` 这里需要我们传入的shape的值。

In [8]:
def ones(shape):
    
    ones = tf.ones(shape)
    
    with tf.Session() as sess:
        ones = sess.run(ones)
        
    return ones

def zeros(shape):
    
    zeros = tf.zeros(shape)
    
    with tf.Session() as sess:
        zeros = sess.run(zeros)
        
    return zeros

In [9]:
print ("ones = " + str(ones([3])))
print ("zeros = " + str(zeros([3])))

ones = [1. 1. 1.]
zeros = [0. 0. 0.]
